# 개인프로젝트
###  #01.프로젝트 개요

## 관심사
- 관심있는 브랜드의 남성 스포츠 용품에 대한 정보 수집 : 미즈노, 아디다스 


        미즈노 남성 스포츠용품 웹페이지 url : https://kor.mizuno.com/kr-kr/044/?start=0&sz=12

        아디다스 남성 스포츠 용품 웹페이지 url : https://www.adidas.co.kr/men-performance?start=0


- 미즈노 크롤링 하나 데이터프레임+ 아디다스 크롤링 데이터 프레임 하나로  합치기 


In [1]:
# 필요한 모듈 참조
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame    

import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time


#### #02.웹 페이지 코드 수집 - 미즈노부터 ㄱㄱ 


미즈노는 각 페이지 별로 상품 수집해서 그 전에는 이미지링크 + 상품 상세보기 링크 + 상품이름 -> 링크를 타고 들어가자 
그다음에 정가, 할인가 + 리뷰 수 

In [2]:



chromedriver_autoinstaller.install()
driver = webdriver.Chrome()
driver.implicitly_wait(5)

In [3]:

## 수집할 컨텐츠가 있는 웹페이지의 주소 

## https://kor.mizuno.com/kr-kr/044/  
## https://kor.mizuno.com/kr-kr/044/?start=0&sz=12

## 페이지마다 url주소의 start 값이 12씩 늘어남
mizuno_num =[]
for i in range(0,34) :
    mizuno_num.append(i*12)
print(mizuno_num)



[0, 12, 24, 36, 48, 60, 72, 84, 96, 108, 120, 132, 144, 156, 168, 180, 192, 204, 216, 228, 240, 252, 264, 276, 288, 300, 312, 324, 336, 348, 360, 372, 384, 396, 408]


#### #03. 응답결과로부터 내용 추출하기

In [30]:
## 결과를 수집할 리스트 지정 (딕셔너리를 원소로 갖는 리스트)
## 미즈노 빈 list
mizuno_result =[]
# 상품리스트가 나온 페이지에서 
# 링크 + 상품이름 + 이미지링크 + 상품 상세보기 추출
for number in mizuno_num :
    url = "https://kor.mizuno.com/kr-kr/044/?start=%d&sz=12" % number
    # url 접속 
    driver.get(url)
    time.sleep(1)

    ## 전체 HTML 파일
    soup = BeautifulSoup(driver.page_source)
    mizuno_list = soup.select("#search-result-items .product-tile")
    print("페이지: %d" % number+1)
    result_dict = {}
    for i, mizuno in enumerate(mizuno_list):
        # print("-" * 50)
        # print("%d번째 상품" % i)

        # 상품이름
        mizuno_nameEl= mizuno.select(".name-link")
        mizuno_name = mizuno_nameEl[0].text.strip()
        # print(mizuno_name)

        ## 이미지 링크 
        mizuno_imageEl = mizuno.select(".thumb-link > img")
        mizuno_image = mizuno_imageEl[0].attrs['src']
        # print(mizuno_image)
        ## 상세보기 링크
        mizuno_detailEl = mizuno.select(".product-name > .name-link")
        mizuno_detail = mizuno_detailEl[0].attrs['href']
        # print(mizuno_detail)
        # ---------------------------------------------------
            
        #### 상세보기 수집 (시작) 

        # --------------mizuno_detail-------------------------------------
            
        driver.get(mizuno_detail)
        time.sleep(1)
        ## 상세보기 HTML            
        detailSoup = BeautifulSoup(driver.page_source)
        # print(detailSoup)

        # 정가, 할인금액,최종금액 + 리뷰 수 추출
        detailEl = detailSoup.select(".pt_product-details")
        # print(detailEl)
        for detail in detailEl :
            standardPriceEl =detail.select("#product-content > .product-price > span "
            if len(standardPriceEl) ==1 :
                LastPrice = standardPriceEl[0].text.strip()
                standardPrice = LastPrice
                discountPrice = None
                # print(LastPrice)      
            elif len(standardPriceEl) == 3 :
                standardPrice = standardPriceEl[0].text.strip()
                LastPrice = standardPriceEl[1].text.strip()
                discountPrice = standardPriceEl[2].text.strip()
            reviewEl = detail.select(".naverReview > h2")
            
            
            # ---------------------------------------------------
            
            #### 상세보기 수집 (끝) 

            # --------------mizuno_detail-------------------------------------
            
            ## 수집결과를 딕셔너리에 저장
            result_dict = {"상품이름":mizuno_name,"정가":standardPrice,"할인 금액":discountPrice,
                           "최종 가격 ":LastPrice,"이미지 링크": mizuno_image}
            ## 최종 결과 리스트에 추가
            mizuno_result.append(result_dict)
df = DataFrame(mizuno_result)


상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 12
상품수: 11
상품수: 0


In [31]:
## 엑셀파일로 변환 + 데이터 형식 통일 시켜보기 
df


,상품이름,정가,할인 금액,최종 가격,이미지 링크
0,WAVE REBELLION PRO_J1GC2317,"239,000원",None,"239,000원",https://kor.mizuno.com/dw/image/v2/BBVP_PRD/on...
1,MORELIA SALA JAPAN TF_Q1GB2100,"219,000원","(할인금액: 21,900\n원)","197,100원",https://kor.mizuno.com/dw/image/v2/BBVP_PRD/on...
2,MIZUNO ΑLPHA JAPAN_P1GA2360,"309,000원","(할인금액: 30,900\n원)","278,100원",https://kor.mizuno.com/dw/image/v2/BBVP_PRD/on...
3,MIZUNO ΑLPHA JAPAN AG_P1GA2361,"289,000원","(할인금액: 28,900\n원)","260,100원",https://kor.mizuno.com/dw/image/v2/BBVP_PRD/on...
4,WAVE RIDER 10_D1GA2104,"159,000원",None,"159,000원",https://kor.mizuno.com/dw/image/v2/BBVP_PRD/on...
...,...,...,...,...,...
402,ESSENTIAL SUN CAP_33YW2305,"35,000원",None,"35,000원",https://kor.mizuno.com/dw/image/v2/BBVP_PRD/on...
403,STRETCH JOGGER PANTS_32YD3026,"99,000원",None,"99,000원",https://kor.mizuno.com/dw/image/v2/BBVP_PRD/on...
404,WAVE DRIVE 9_81GA2205,"139,000원",None,"139,000원",https://kor.mizuno.com/dw/image/v2/BBVP_PRD/on...
405,WAVE REVOLT 3_J1GC2381,"99,000원",None,"99,000원",https://kor.mizuno.com/dw/image/v2/BBVP_PRD/on...
